# Assessment of Microservice Architecture Deployment Alternatives by Automated Performance Testing

Read Data from Fanban: configuration, mean vector, SD vector,  mix

In [1]:
dataFile<-read.csv("FanbanData.csv", header= TRUE,sep=",")

Read operational profile  data (e.g., from NetFlix). Change / remove scale factor if needed

In [2]:
inputFile<-read.csv("OperationalProfileData.csv", header= TRUE,sep=",")
usersLoad<-inputFile[,1]
accessCount<-inputFile[,2]
scaleFactor<-2.5
usersLoadScaled<-floor(scaleFactor*usersLoad)
accessFrequency<-accessCount/sum(accessCount)
#print(inputFile)

Sum up frequency every 50 users

In [3]:
byFifty<-which(usersLoadScaled %% 50 == 0)
binProb<-c()
for(i in 1:length(byFifty)){
	if(i==1){
		binProb[i]<-sum(accessFrequency[1:byFifty[i]])
	}else{
		binProb[i]<-sum(accessFrequency[(byFifty[i-1]+1):byFifty[i]])
	}
}

Extract settings of resources: ID, Users, Memory, CPU, Carts Replicas

In [4]:
mySettings<-as.data.frame(matrix(nrow=nrow(dataFile)%/% 3, ncol=5))
for(i in 0:((nrow(dataFile)%/% 3)-1)){
    mySettings[i+1,1:5]<-dataFile[1+3*i,1:5]
}
colnames(mySettings)<-colnames(dataFile[,1:5])
noMicroServices<-ncol(dataFile)-6
#print(mySettings)

Process time data in seconds for benchmark workload situation:number of users =2; 

In [5]:
avgVectorB<-data.frame(c(mySettings[1,],dataFile[1,7:(6+noMicroServices)]))
SDVectorB<-data.frame(c(mySettings[1,],dataFile[2,7:(6+noMicroServices)]))
threshold<-data.frame(c(mySettings[1,],avgVectorB[1,6:(5+noMicroServices)]+3*SDVectorB[1,6:(5+noMicroServices)]))
mixB<-data.frame(c(mySettings[1,],dataFile[3,7:(6+noMicroServices)]))

Process time data in seconds per workload situation \lambda

In [6]:
avgVector<-as.data.frame(matrix(nrow=nrow(mySettings)-1, ncol=ncol(dataFile)-1))
for(i in 1:(nrow(mySettings)-1)){
	avgVector[i,]<-data.frame(c(mySettings[i+1,],dataFile[1+3*i,7:(6+noMicroServices)]))
}
colnames(avgVectorB)<-c(colnames(dataFile[-6]))
SDVector<-as.data.frame(matrix(nrow=nrow(mySettings)-1, ncol=ncol(dataFile)-1))
for(i in 1:(nrow(mySettings)-1)){
	SDVector[i,]<-data.frame(c(mySettings[i+1,],dataFile[2+3*i,7:(6+noMicroServices)]))
}
colnames(SDVectorB)<-c(colnames(dataFile[-6]))

mixTemp<-as.data.frame(matrix(nrow=nrow(mySettings)-1, ncol=ncol(dataFile)-1))
for(i in 1:(nrow(mySettings)-1)){
	mixTemp[i,]<-data.frame(c(mySettings[i+1,],dataFile[3+3*i,7:(6+noMicroServices)]))
}
colnames(mixTemp)<-c(colnames(dataFile[-6]))
#print(mySettings)

Check pass criterion and compute relativeMass

In [7]:
passCriteria<-as.data.frame(matrix(nrow=nrow(mySettings)-1, ncol=ncol(dataFile)-1))
for(i in 1:(nrow(mySettings)-1)){
	passCriteria[i,]<-data.frame(c(mySettings[i+1,],avgVector[i,6:(5+noMicroServices)]))
}
relativeMass<-c()
colnames(passCriteria)<-c(colnames(dataFile[-6]))
mix<-as.data.frame(matrix(nrow=nrow(mySettings)-1, ncol=ncol(dataFile)-1))
for(j in 1:nrow(passCriteria)){
	mix[j,]<-mixTemp[j,]
	for(i in 6:(5+noMicroServices)){
		if(passCriteria[j,i]>threshold[i]) 
			mix[j,i]<-0
	}
	relativeMass[j]<-sum(mix[j,6:(5+noMicroServices)])
}

Define a matrix of state frequencies every fifty user

In [8]:
aggregatedValues<-matrix(c(usersLoadScaled[byFifty], binProb), ncol=2,nrow=6, dimnames=list(c(1,2,3,4,5,6), c("Users", "Aggregated Mass")))

Compute absoluteRelativeMass

In [9]:
finalSettings<-mySettings
relativeMass<-c(0, relativeMass)
finalSettings$relativeMass<-relativeMass
absoluteRelativeMass<-c()
for(j in 1:nrow(finalSettings)){	
	absoluteRelativeMass[j]<-finalSettings[j,"relativeMass"]*aggregatedValues[match(finalSettings[j,"Users"], aggregatedValues[,1]),2]
}
absoluteRelativeMass[1]<-0
finalSettings$absoluteRelativeMass<-absoluteRelativeMass

Create the datasets for each plot and plot the datasets as in Fig. 3, a)

In [10]:
mySettingsUnique<-unique(mySettings[3:5])
set<-list()
ordSet<-list()
for(i in 1:nrow(mySettingsUnique)){
	set[[i]]<-finalSettings[which(finalSettings[,3] == mySettingsUnique[i,1]&finalSettings[,4] == mySettingsUnique[i,2]&finalSettings[,5] == mySettingsUnique[i,3]),]
    ordSet[[i]]<-set[[i]][,c(2,7)][order(set[[i]][,c(2,7)][,1]),]
}

#if(match(mySettingsUnique[2,1],finalSettings[,3])& match(mySettingsUnique[2,2],finalSettings[,4])& match(mySettingsUnique[2,3],finalSettings[,5])){
#	match(mySettingsUnique[3,],finalSettings[,3])
#}
#print(mySettingsUnique)
#print(finalSettings)

Plot curves

In [11]:
plot(aggregatedValues, xlim=c(50, 300), ylim=c(0, 0.3))
polygon(c(50,aggregatedValues[,1],300),c(0,aggregatedValues[,2],0), col="lightblue", lty = 2, lwd = 2, border = "blue")
for(i in 1:nrow(mySettingsUnique)){
    lines(ordSet[[i]], type="l", col="blue")	
    polygon(c(50,t(ordSet[[i]][1]),300),c(0,t(ordSet[[i]][2]),0) , col=rgb(.5,.4,.1,.30), lty = 2, lwd = 2, border = "black")
}
#graphics.off()